# Read the data

Stack Overflow’s annual Developer Survey is a large and comprehensive survey of people who code around the world. Each year, they field a survey covering everything from developers’ favorite technologies to their job preferences. Download the data from https://www.kaggle.com/aitzaz/stack-overflow-developer-survey-2020.

In [1]:
import pandas as pd
import numpy as np
import zipfile

zf = zipfile.ZipFile('stack_overflow_2020.zip') 
df = pd.read_csv(zf.open('developer_survey_2020/survey_results_public.csv'))

columns_to_keep = ['Respondent', 'Country', 'Employment', 'Ethnicity', 'Gender', 'YearsCode', 'EdLevel', 'JobSat', 'LanguageWorkedWith', 'Age']
df = df.filter(items=columns_to_keep)

# Glimpse the data

In [2]:
print(df.shape)
df.head()

(64461, 10)


,Respondent,Country,Employment,Ethnicity,Gender,YearsCode,EdLevel,JobSat,LanguageWorkedWith,Age
0,1,Germany,"Independent contractor, freelancer, or self-em...",White or of European descent,Man,36,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Slightly satisfied,C#;HTML/CSS;JavaScript,NaN
1,2,United Kingdom,Employed full-time,NaN,NaN,7,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Very dissatisfied,JavaScript;Swift,NaN
2,3,Russian Federation,NaN,NaN,NaN,4,NaN,NaN,Objective-C;Python;Swift,NaN
3,4,Albania,NaN,White or of European descent,Man,7,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Slightly dissatisfied,NaN,25.0
4,5,United States,Employed full-time,White or of European descent,Man,15,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",NaN,HTML/CSS;Ruby;SQL,31.0


# Check Dtypes

In [3]:
df.dtypes

Respondent              int64
Country                object
Employment             object
Ethnicity              object
Gender                 object
YearsCode              object
EdLevel                object
JobSat                 object
LanguageWorkedWith     object
Age                   float64
dtype: object

In [4]:
df.YearsCode.unique()

array(['36', '7', '4', '15', '6', '17', '8', '10', '35', '5', '37', '19',
       '9', '22', '30', '23', '20', '2', 'Less than 1 year', '3', '13',
       '25', '16', '43', '11', '38', '33', nan, '24', '21', '12', '40',
       '27', '50', '46', '14', '18', '28', '32', '44', '26', '42', '31',
       '34', '29', '1', '39', '41', '45', 'More than 50 years', '47',
       '49', '48'], dtype=object)

In [5]:
df['YearsCode'] = df['YearsCode'].replace('Less than 1 year', 0)
df['YearsCode'] = df['YearsCode'].replace('More than 50 years', 50)
df['YearsCode'] = df['YearsCode'].astype('float64')

In [6]:
df.dtypes

Respondent              int64
Country                object
Employment             object
Ethnicity              object
Gender                 object
YearsCode             float64
EdLevel                object
JobSat                 object
LanguageWorkedWith     object
Age                   float64
dtype: object

# Check missing data

In [7]:
df.isnull().sum()

Respondent                0
Country                 389
Employment              607
Ethnicity             18513
Gender                13904
YearsCode              6777
EdLevel                7030
JobSat                19267
LanguageWorkedWith     7083
Age                   19015
dtype: int64

# Categoricals

In [8]:
age_bins = [0, 18, 24, 34, 44, 54, 64, 100]

age_labels = ['17 years or younger', '18 - 24 years old', '25 - 34 years old', '35 - 44 years old', '45 - 54 years old', '55 - 64 years old', '65 years or older']

df['age_cat'] = pd.cut(df['Age'], 
                        bins=age_bins,
                        labels=age_labels,
                        right=True)

In [9]:
df[df.Age.notnull()][['Age', 'age_cat']]

,Age,age_cat
3,25.0,25 - 34 years old
4,31.0,25 - 34 years old
7,36.0,35 - 44 years old
8,30.0,25 - 34 years old
9,22.0,18 - 24 years old
...,...,...
64426,22.0,18 - 24 years old
64427,34.0,25 - 34 years old
64430,38.0,35 - 44 years old
64433,64.0,55 - 64 years old


<b>Number of programmers in each bucket<b>

In [10]:
df.age_cat.value_counts()

25 - 34 years old      20866
18 - 24 years old      10026
35 - 44 years old       8674
45 - 54 years old       2807
17 years or younger     1844
55 - 64 years old        957
65 years or older        271
Name: age_cat, dtype: int64

# Clean labels

In [11]:
df.EdLevel.unique()

array(['Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
       'Bachelor’s degree (B.A., B.S., B.Eng., etc.)', nan,
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       'Professional degree (JD, MD, etc.)',
       'Some college/university study without earning a degree',
       'Associate degree (A.A., A.S., etc.)',
       'Other doctoral degree (Ph.D., Ed.D., etc.)',
       'Primary/elementary school',
       'I never completed any formal education'], dtype=object)

In [12]:
df['EdLevel'] = df['EdLevel'].str.replace(r"\(.*\)", "")
df.EdLevel.unique()

array(['Master’s degree ', 'Bachelor’s degree ', nan, 'Secondary school ',
       'Professional degree ',
       'Some college/university study without earning a degree',
       'Associate degree ', 'Other doctoral degree ',
       'Primary/elementary school',
       'I never completed any formal education'], dtype=object)

<b>Years coding and Age for each education level<b>

In [13]:
df[['EdLevel', 'YearsCode', 'Age']].groupby('EdLevel').mean().sort_values(by='YearsCode', ascending=False)

,YearsCode,Age
EdLevel,,
Other doctoral degree,21.367754,39.428680
Master’s degree,15.207951,33.721334
Professional degree,15.177835,34.743719
Some college/university study without earning a degree,12.892907,30.825112
Associate degree,12.556593,31.516227
Bachelor’s degree,12.048921,30.431446
I never completed any formal education,9.667431,29.951220
Secondary school,8.162622,23.972902
Primary/elementary school,6.584158,20.433333


# Melt

In [14]:
df['LanguageWorkedWith'].unique()

array(['C#;HTML/CSS;JavaScript', 'JavaScript;Swift',
       'Objective-C;Python;Swift', ...,
       'Bash/Shell/PowerShell;C;C++;Go;HTML/CSS;JavaScript;Kotlin;Python;Ruby;Rust;SQL',
       'Bash/Shell/PowerShell;C#;C++;HTML/CSS;JavaScript;Kotlin;PHP;SQL;VBA',
       'Bash/Shell/PowerShell;C;C#;C++;HTML/CSS;Java;JavaScript;Objective-C;Python;Swift'],
      dtype=object)

<b>1) Create a column for each programming language in existence<b>

In [15]:
# Extracting Languages
languages = []
for text in df['LanguageWorkedWith'].dropna().values.tolist():
    temp = text.split(';')
    for lang in temp:
        if lang not in languages:
            languages.append(lang.strip())
            
print(languages)

['C#', 'HTML/CSS', 'JavaScript', 'Swift', 'Objective-C', 'Python', 'Ruby', 'SQL', 'Java', 'PHP', 'C', 'TypeScript', 'Bash/Shell/PowerShell', 'Kotlin', 'R', 'VBA', 'Perl', 'Scala', 'C++', 'Go', 'Haskell', 'Rust', 'Dart', 'Julia', 'Assembly']


In [16]:
for lang in languages:
    df[lang] = np.where(df['LanguageWorkedWith'].str.contains(lang, na=False, regex=False), True, False)

df.head()

,Respondent,Country,Employment,Ethnicity,Gender,YearsCode,EdLevel,JobSat,LanguageWorkedWith,Age,...,VBA,Perl,Scala,C++,Go,Haskell,Rust,Dart,Julia,Assembly
0,1,Germany,"Independent contractor, freelancer, or self-em...",White or of European descent,Man,36.0,Master’s degree,Slightly satisfied,C#;HTML/CSS;JavaScript,NaN,...,False,False,False,False,False,False,False,False,False,False
1,2,United Kingdom,Employed full-time,NaN,NaN,7.0,Bachelor’s degree,Very dissatisfied,JavaScript;Swift,NaN,...,False,False,False,False,False,False,False,False,False,False
2,3,Russian Federation,NaN,NaN,NaN,4.0,NaN,NaN,Objective-C;Python;Swift,NaN,...,False,False,False,False,False,False,False,False,False,False
3,4,Albania,NaN,White or of European descent,Man,7.0,Master’s degree,Slightly dissatisfied,NaN,25.0,...,False,False,False,False,False,False,False,False,False,False
4,5,United States,Employed full-time,White or of European descent,Man,15.0,Bachelor’s degree,NaN,HTML/CSS;Ruby;SQL,31.0,...,False,False,False,False,False,False,False,False,False,False


In [17]:
df.iloc[0]

Respondent                                                               1
Country                                                            Germany
Employment               Independent contractor, freelancer, or self-em...
Ethnicity                                     White or of European descent
Gender                                                                 Man
YearsCode                                                               36
EdLevel                                                   Master’s degree 
JobSat                                                  Slightly satisfied
LanguageWorkedWith                                  C#;HTML/CSS;JavaScript
Age                                                                    NaN
age_cat                                                                NaN
C#                                                                    True
HTML/CSS                                                              True
JavaScript               

<b>2) Pandas melt()<b>

In [18]:
melt_df = df.melt(id_vars=columns_to_keep, value_vars=languages, var_name='language', value_name='language_value')

In [19]:
melt_df[melt_df.Respondent == 1].head()

,Respondent,Country,Employment,Ethnicity,Gender,YearsCode,EdLevel,JobSat,LanguageWorkedWith,Age,language,language_value
0,1,Germany,"Independent contractor, freelancer, or self-em...",White or of European descent,Man,36.0,Master’s degree,Slightly satisfied,C#;HTML/CSS;JavaScript,NaN,C#,True
64461,1,Germany,"Independent contractor, freelancer, or self-em...",White or of European descent,Man,36.0,Master’s degree,Slightly satisfied,C#;HTML/CSS;JavaScript,NaN,HTML/CSS,True
128922,1,Germany,"Independent contractor, freelancer, or self-em...",White or of European descent,Man,36.0,Master’s degree,Slightly satisfied,C#;HTML/CSS;JavaScript,NaN,JavaScript,True
193383,1,Germany,"Independent contractor, freelancer, or self-em...",White or of European descent,Man,36.0,Master’s degree,Slightly satisfied,C#;HTML/CSS;JavaScript,NaN,Swift,False
257844,1,Germany,"Independent contractor, freelancer, or self-em...",White or of European descent,Man,36.0,Master’s degree,Slightly satisfied,C#;HTML/CSS;JavaScript,NaN,Objective-C,False


<b>Figure out the most commonly known programming languages<b>

In [20]:
melt_df = melt_df.loc[melt_df['language_value'] == True]
melt_df[['language', 'language_value']].groupby(['language']).count().sort_values(by=['language_value'], ascending=False)

,language_value
language,
C,46769
Java,45749
JavaScript,38822
HTML/CSS,36181
SQL,31413
Python,25287
Bash/Shell/PowerShell,18980
C#,18041
PHP,15007


<b>Programing languages are used by people very satisfied with their jobs<b>

In [22]:
job_sat = melt_df.groupby(['JobSat', 'language']).size().to_frame('count')
job_sat

count
JobSat                             language                    
Neither satisfied nor dissatisfied Assembly                 245
                                   Bash/Shell/PowerShell   1645
                                   C                       4290
                                   C#                      1770
                                   C++                     1083
...                                                         ...
Very satisfied                     SQL                     7911
                                   Scala                    590
                                   Swift                    871
                                   TypeScript              4203
                                   VBA                      821

[125 rows x 1 columns]

In [23]:
job_sat.loc['Very satisfied'].sort_values(by='count', ascending=False)

,count
language,
C,11175
Java,10986
JavaScript,9619
HTML/CSS,8671
SQL,7911
Python,6039
Bash/Shell/PowerShell,5435
C#,4537
TypeScript,4203
